In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the pre-trained ResNet-50 model
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')


2024-01-06 10:33:36.539497: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


94765736/94765736 [==============================] - 3s 0us/step


In [4]:
def prepare_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

# Prepare the images
img1 = prepare_image('./Downloads/profile_PIX_.JPG')
img2 = prepare_image('./Downloads/image_.jpeg')


In [5]:
# Extract embeddings
embedding1 = model.predict(img1)
embedding2 = model.predict(img2)


1/1 [==============================] - 0s 155ms/step


In [7]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute similarity between the embeddings
similarity = cosine_similarity(embedding1, embedding2)
print(f"Similarity: {similarity}")


Similarity: [[0.62556916]]


In [12]:
# Prepare and extract embedding for a test image (another instance of img1)
test_img = prepare_image('./Downloads/IMG-0316.jpg')
test_embedding = model.predict(test_img)

# Compute similarity with img1
test_similarity = cosine_similarity(test_embedding, embedding1)
print(f"Test Similarity: {test_similarity}")


1/1 [==============================] - 0s 142ms/step
Test Similarity: [[0.6281172]]


In [6]:
import face_recognition

# Load the images into numpy arrays
img1 = face_recognition.load_image_file('./Downloads/profile_PIX_.JPG')
img2 = face_recognition.load_image_file('./Downloads/img_test2.jpeg')

# Get the face encodings for each image
img1_encoding = face_recognition.face_encodings(img1)[0]
img2_encoding = face_recognition.face_encodings(img2)[0]

# Compare the faces and return the result
results = face_recognition.compare_faces([img1_encoding], img2_encoding)

if results[0]:
    print("It's a match!")
else:
    print("It's not a match.")


IndexError: list index out of range

In [14]:
from deepface import DeepFace

# Verifying two images
result = DeepFace.verify(img1_path = './Downloads/profile_PIX_.JPG', img2_path = './Downloads/IMG_3395.jpg')

print("Are these images of the same person?: ", result["verified"])


Are these images of the same person?:  False


In [15]:
from deepface import DeepFace
result = DeepFace.verify(img1_path = './Downloads/profile_PIX_.JPG', img2_path = './Downloads/IMG_3395.jpg',model_name = "VGG-Face")

In [16]:
print("Are these images of the same person?: ", result["verified"])


Are these images of the same person?:  False


In [12]:
from deepface import DeepFace

# Analyzing an image
analysis = DeepFace.analyze(img_path = './Downloads/profile_PIX_.JPG', actions = ['age', 'gender', 'race', 'emotion'])

print(analysis)


24-01-06 14:24:47 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /Users/mac/.deepface/weights/facial_expression_model_weights.h5
100%|█████████████████████████████████████████████████████| 5.98M/5.98M [00:00<00:00, 50.8MB/s]


24-01-06 14:24:51 - age_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5
To: /Users/mac/.deepface/weights/age_model_weights.h5
100%|███████████████████████████████████████████████████████| 539M/539M [00:16<00:00, 32.4MB/s]


24-01-06 14:25:13 - gender_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/gender_model_weights.h5
To: /Users/mac/.deepface/weights/gender_model_weights.h5
100%|███████████████████████████████████████████████████████| 537M/537M [00:16<00:00, 31.8MB/s]


24-01-06 14:25:37 - race_model_single_batch.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/race_model_single_batch.h5
To: /Users/mac/.deepface/weights/race_model_single_batch.h5
100%|███████████████████████████████████████████████████████| 537M/537M [00:16<00:00, 32.3MB/s]
Action: emotion: 100%|███████████████████████████████████████████| 4/4 [00:07<00:00,  1.77s/it]

[{'age': 24, 'region': {'x': 834, 'y': 1336, 'w': 1540, 'h': 1540}, 'face_confidence': 7.716838093590923, 'gender': {'Woman': 0.0003259293180235545, 'Man': 99.99967813491821}, 'dominant_gender': 'Man', 'race': {'asian': 2.026354639639294e-08, 'indian': 3.8975400684648775e-06, 'black': 100.0, 'white': 7.09193446156442e-12, 'middle eastern': 3.185289421006425e-12, 'latino hispanic': 2.0148016588450446e-07}, 'dominant_race': 'black', 'emotion': {'angry': 0.00826929826871492, 'disgust': 4.580646972840441e-08, 'fear': 0.00042392071009089705, 'happy': 1.0089879855513573, 'sad': 0.0895503384526819, 'surprise': 0.011559346603462473, 'neutral': 98.88120889663696}, 'dominant_emotion': 'neutral'}]


In [17]:
# Capture video from the first webcam connected to your computer.
video_capture = cv2.VideoCapture(0)


In [ ]:
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    if not ret:
        break  # If there's an error or no frame, exit the loop

    # Analyze the frame with DeepFace
    try:
        # This will analyze the facial attributes such as emotion, age, gender, and race
        # You can reduce the actions to speed up the process if needed
        analysis = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        print(analysis)

        # Use the information in 'analysis' to draw boxes or text on the frame
        # For instance, here's how you might display the emotion:
        emotion = analysis["dominant_emotion"]
        cv2.putText(frame, emotion, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)

    except Exception as e:
        print(e)  # Print any errors

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything's done, release the capture
video_capture.release()
cv2.destroyAllWindows()


[{'emotion': {'angry': 3.5866968049817626e-05, 'disgust': 6.164990335889315e-12, 'fear': 91.3627992435933, 'happy': 3.6878052428000876, 'sad': 0.3756285662944834, 'surprise': 3.1522300061905377, 'neutral': 1.4214934949286586}, 'dominant_emotion': 'fear', 'region': {'x': 0, 'y': 0, 'w': 1280, 'h': 720}, 'face_confidence': 0}]
list indices must be integers or slices, not str
[{'emotion': {'angry': 2.9239787902481586e-05, 'disgust': 4.215516796311407e-12, 'fear': 85.69010666726749, 'happy': 4.309787087465872, 'sad': 0.17879809216954207, 'surprise': 8.475015792248751, 'neutral': 1.3462630358322816}, 'dominant_emotion': 'fear', 'region': {'x': 0, 'y': 0, 'w': 1280, 'h': 720}, 'face_confidence': 0}]
list indices must be integers or slices, not str
[{'emotion': {'angry': 3.428246646447274e-05, 'disgust': 6.640433938264007e-12, 'fear': 57.52282649006528, 'happy': 3.671013498781502, 'sad': 0.08672985262105136, 'surprise': 37.57894187224604, 'neutral': 1.1404534373636916}, 'dominant_emotion': 'f

In [2]:
from deepface import DeepFace
import cv2

video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    if not ret:
        break  # If there's an error or no frame, exit the loop

    # Save the captured frame to a file
    frame_path = 'current_frame.jpg'
    cv2.imwrite(frame_path, frame)

    try:
        # Perform real-time face verification
        result = DeepFace.verify(img1_path='./Downloads/profile_PIX_.JPG', img2_path=frame_path, enforce_detection=False)

        # Print the result to understand its structure
        print(result)

        # Proceed only if verification was successful and the required data exists
        if result["verified"]:
            # Check if 'img2_region' exists in the result before accessing it
            if "img2" in result:
                # Draw a rectangle around the face and tag as 'Found'
                face_coordinates = result["img2"]  # Coordinates of the face in the frame
                cv2.rectangle(frame, (face_coordinates[0], face_coordinates[1]), (face_coordinates[2], face_coordinates[3]), (0, 255, 0), 2)
                cv2.putText(frame, "Thats me!!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            else:
                print("img2_region not found in the result.")
        else:
            cv2.putText(frame, "Not Found", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    except Exception as e:
        print(e)  # Print any errors

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything's done, release the capture
video_capture.release()
cv2.destroyAllWindows()


{'verified': False, 'distance': 0.5289615420480848, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 834, 'y': 1336, 'w': 1540, 'h': 1540}, 'img2': {'x': 0, 'y': 0, 'w': 1280, 'h': 720}}, 'time': 5.51}
{'verified': False, 'distance': 0.541817496754095, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 834, 'y': 1336, 'w': 1540, 'h': 1540}, 'img2': {'x': 0, 'y': 0, 'w': 1280, 'h': 720}}, 'time': 5.02}
{'verified': False, 'distance': 0.5199414249586043, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 834, 'y': 1336, 'w': 1540, 'h': 1540}, 'img2': {'x': 0, 'y': 0, 'w': 1280, 'h': 720}}, 'time': 4.67}
{'verified': False, 'distance': 0.5220852285217006, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine

In [ ]:
{'verified': True, 'distance': 0.1607575105175798, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 834, 'y': 1336, 'w': 1540, 'h': 1540}, 'img2': {'x': 716, 'y': 238, 'w': 306, 'h': 306}}, 'time': 4.89}

In [16]:
from deepface import DeepFace
import cv2

# Load OpenCV's Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    if not ret:
        break  # If there's an error or no frame, exit the loop

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Save the captured frame to a file
        face_frame = frame[y:y+h, x:x+w]
        cv2.imwrite('current_frame.jpg', face_frame)

        try:
            # Perform real-time face verification
            result = DeepFace.verify(img1_path='./Downloads/profile_PIX_.JPG', img2_path='current_frame.jpg', enforce_detection=False)
            print(result)
            if result["verified"]:
                # Draw a rectangle around the face and tag as 'Thats me!!'
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame, "Thats me!!", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        except Exception as e:
            print(e)  # Print any errors

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything's done, release the capture
video_capture.release()
cv2.destroyAllWindows()


{'verified': False, 'distance': 0.5705734559850948, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 834, 'y': 1336, 'w': 1540, 'h': 1540}, 'img2': {'x': 0, 'y': 0, 'w': 183, 'h': 183}}, 'time': 5.26}
{'verified': False, 'distance': 0.6346981148054035, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 834, 'y': 1336, 'w': 1540, 'h': 1540}, 'img2': {'x': 0, 'y': 0, 'w': 212, 'h': 212}}, 'time': 6.97}
{'verified': False, 'distance': 0.5142694949828077, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 834, 'y': 1336, 'w': 1540, 'h': 1540}, 'img2': {'x': 0, 'y': 0, 'w': 252, 'h': 252}}, 'time': 5.36}
{'verified': False, 'distance': 0.47788070128750904, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine'

KeyboardInterrupt: 

In [ ]:
from deepface import DeepFace
import cv2

# Load a pre-trained face detection model from OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    if not ret:
        break  # If there's an error or no frame, exit the loop

    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    # Save the captured frame to a file
    frame_path = 'current_frame.jpg'
    cv2.imwrite(frame_path, frame)

    try:
        # Perform real-time face verification
        result = DeepFace.verify(img1_path='./Downloads/profile_PIX_.JPG', img2_path=frame_path, enforce_detection=False)
        print(result)
        # Handle verification result
        for (x, y, w, h) in faces:
            if result["verified"]:
                # Draw a rectangle around the face and tag as 'That's me!!'
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame, "That's me!!", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                # Display similarity score and distance
                cv2.putText(frame, f"Score: {result['score']:.2f}", (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                cv2.putText(frame, f"Distance: {result['distance']:.2f}", (x, y+h+40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            else:
                # If the face is not verified, tag as 'New user'
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
                cv2.putText(frame, "New patient", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                # Display similarity score and distance
                cv2.putText(frame, f"Score: {result['score']:.2f}", (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                cv2.putText(frame, f"Distance: {result['distance']:.2f}", (x, y+h+40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    except Exception as e:
        print(e)  # Print any errors

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything's done, release the capture
video_capture.release()
cv2.destroyAllWindows()


In [17]:
from deepface import DeepFace
import cv2

# Load a pre-trained face detection model from OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

video_capture = cv2.VideoCapture(0)

# Define a threshold for the face distance for your specific scenario
threshold = 0.45

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    if not ret:
        break  # If there's an error or no frame, exit the loop

    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    # Save the captured frame to a file
    frame_path = 'current_frame.jpg'
    cv2.imwrite(frame_path, frame)

    try:
        # Perform real-time face verification
        result = DeepFace.verify(img1_path='./Downloads/profile_PIX_.JPG', img2_path=frame_path, enforce_detection=False)
        print(result)

        for (x, y, w, h) in faces:
            # Initialize label and color (default to 'New user' and red)
            label = "New user"
            color = (0, 0, 255)  # Red for 'New user'

            # Check the result and update label and color if it's a match
            if result.get("distance") is not None and result["distance"] <= threshold:
                label = "That's me!!"
                color = (0, 255, 0)  # Green for 'That's me!!'

            # Draw a rectangle and label with the determined color
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

            # Display distance if available
            if result.get("distance") is not None:
                cv2.putText(frame, f"Distance: {result['distance']:.2f}", (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    except Exception as e:
        print(e)  # Print any errors

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything's done, release the capture
video_capture.release()
cv2.destroyAllWindows()


{'verified': False, 'distance': 0.5227662752645965, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 834, 'y': 1336, 'w': 1540, 'h': 1540}, 'img2': {'x': 0, 'y': 0, 'w': 1280, 'h': 720}}, 'time': 6.01}
{'verified': False, 'distance': 0.4505313111874335, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 834, 'y': 1336, 'w': 1540, 'h': 1540}, 'img2': {'x': 639, 'y': 274, 'w': 223, 'h': 223}}, 'time': 4.54}
{'verified': False, 'distance': 0.4980159507057351, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 834, 'y': 1336, 'w': 1540, 'h': 1540}, 'img2': {'x': 0, 'y': 0, 'w': 1280, 'h': 720}}, 'time': 5.11}
{'verified': False, 'distance': 0.497912924447077, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cos

KeyboardInterrupt: 

In [11]:
import face_recognition

image_path = './Downloads/img_test2.jpeg'

# Load the image
image = face_recognition.load_image_file(image_path)

# Find all face locations in the image
face_locations = face_recognition.face_locations(image)

# If no faces are found, face_locations will be an empty list
if not face_locations:
    print("No faces found in the image!")
else:
    # Assuming the first face in the image is the one we want to recognize
    face_encoding = face_recognition.face_encodings(image, known_face_locations=face_locations)[0]
    # Rest of your code...


No faces found in the image!


In [4]:
pip install --upgrade dlib


Note: you may need to restart the kernel to use updated packages.


In [3]:
print(face_encodings)

[]
